In [42]:
# Import necessary libraries

import json
import joblib
from collections import defaultdict
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import scipy.sparse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/agastimhatre/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## Step 1: Join business dataset with reviews dataset. Tokenize reviews and convert to lowercase

In [43]:
# Store reviews for each business
business_information = defaultdict(lambda: {"reviews": []})

with open("yelp_academic_dataset_review.json", "r", encoding="utf-8") as file:

    for line in file: 

        curr = json.loads(line)
        business_information[curr["business_id"]]["reviews"].append(curr["text"])

In [44]:
# Extract business-specific data and store in dictionary
with open("yelp_academic_dataset_business.json", "r", encoding="utf-8") as file:

    for line in file:

        curr = json.loads(line)
        business_information[curr["business_id"]]["name"] = curr["name"]
        business_information[curr["business_id"]]["address"] = curr["address"]
        business_information[curr["business_id"]]["city"] = curr["city"]
        business_information[curr["business_id"]]["state"] = curr["state"]


In [ ]:
# Tokenize reviews and convert all tokens to lowercase
tokenized_business_information = defaultdict(lambda: {"reviews": []})

for id_ in business_information.keys():

    for review in business_information[id_]["reviews"]:

        tokenized_business_information[id_]["reviews"].append([word.lower() for word in word_tokenize(review)])

In [ ]:
# Write joined, tokenized business/review data to a file
with open('business_with_reviews.json', 'w') as f:
    for key, value in tokenized_business_information.items():
        json.dump({key: value}, f)
        f.write('\n')

## Step 2: Output scores for TFIDF ranker

In [45]:
# Read in business reviews from 
business_reviews = dict()

with open('business_with_reviews.json', 'r') as f:

    for line in f:

        record = json.loads(line)
        business_reviews.update(record)

In [50]:
# Test one query on the entire dataset, check results

business_ids = list(business_reviews.keys())
corpus = []
for id_ in business_ids:

    corpus.append(" ".join(" ".join(tokens) for tokens in business_reviews[id_]["reviews"]))

In [ ]:
# Create TFIDF vector and train vectorizer
# Persist vector and model

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(corpus)
joblib.dump(vectorizer, 'tfidf_vectorizer.joblib')
scipy.sparse.save_npz("tfidf_matrix.npz", tfidf_matrix)

In [46]:
# Reinitialize vectorizer and tfidf_matrix if session is lost

vectorizer = joblib.load('tfidf_vectorizer.joblib')
tfidf_matrix = scipy.sparse.load_npz("tfidf_matrix.npz")

In [47]:
query = "cafe with great lattes in san francisco"
query_vec = vectorizer.transform([query])
similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()

In [48]:
scores = []

n = len(business_ids)
for i in range(n): scores.append((business_ids[i], similarities[i]))

scores.sort(key = lambda x: x[1], reverse=True)

In [49]:
for id, _ in scores[:10]:

    print(business_information[id]["name"])

Vico's Mobile Auto Detailing
Francisco's Mobile Auto Detailing
GET Cafe
HOME Page Cafe
Grand Cafe & Bistro
La Mademoiselle Marvelous Pastries and Cafe
The Grind House
Cafe Lavi
Black Press Coffee Shop #003
T cafe


## Step 3: Create/Store Embeddings and Use FAISS to generate scores, Compare to token-matching ranker

In [51]:
model = SentenceTransformer("all-MiniLM-L6-v2")

embeddings = model.encode(corpus, convert_to_numpy=True)
dimension = embeddings.shape[1]

index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [52]:
faiss.write_index(index, "business_index.faiss")

In [ ]:
query = "cafe with great lattes in san francisco"
query_vec = model.encode([query], convert_to_numpy=True)
k = 50
D, I = index.search(query_vec, k)
top_businesses = [business_ids[i] for i in I[0]]

for id in top_businesses:

    print(business_information[id]["name"])

Top results: ['0iHHsSqphxrBWWecYEJcuQ', '9Ekfby4OgzWs96RO8M4tnQ', 'q5Kb63rdNPDJ0MoVtq3KLg', 'qUH2FppY7RL96F5tXzA0hQ', 'vDgT0MyVtWZNX4X9j6EsMg', '_P6AQUbyQltYRcQNrQ8plw', 'gH3fKYUCsjIrIvGgXzXdfA', 'r5--kPhTvamXmN7ZMbUUYQ', 'ngkjYkjJcPQ28gDub3R0IQ', 'AuVDe2eYGHpEmgSMhw0T3Q', 'PC0z5dgURsstoS0-APO4qA', 'xcdT294axowe3Tj-04tHIA', 'uEv1I-0r82Krj4eEysjC4A', 'PCmoZ6i5fXLKti6SmykDNg', 'z2d0Pg8dqLUkrCMVvyYhxw', 'nMn4CTJQPXqpgLltYQrxMA', 'tZU9d-czS4VfY72zE0Homg', 'DQ40RrKUpAra--v6C0Q_2A', 'hzkeaiW4tG0H_nHG31JuVA', 'UxP1VK4lTLKRu9cgjGhdaw', '642kv5mYCqYr6COe49HMzA', 'TqE83vo_rs4h4EycnuLI6g', '-DlF6neBa1HKx7aFi3ZxOg', 'nBr6NgqcZz1GhtAngU0RSQ', 'FRKqZwXnJqPSdtGW-7nwpw', 'cCwGNGOCvhigKOLSFUaVDg', 'eX1eHyJ6iYD1rzfzEox43w', 'VKsqpZVSFgvAkTm5NNU47w', 'h22117eMO-kVtCOLYYL6GA', 'Envz99GhwMkfMg7LzDQeUQ', 'SUpQ7uFtXvdzgVubewMV8g', 'ws0mjFrkanX_sXoUVRh8Uw', 'gXwHMtF-PLNG_9n7t9IxBA', '6F4JadEP6ouUvNiIKr8pMg', 'BUmzDaCXu44GNjvtdePVTQ', 'joCDK7kg-03uo4rZK5X0_A', 'DgF4_72z6UkiGxtPnSJCfg', 'WeU7pN8VmkhxiVD71-pWGA'